In [ ]:
import os
import pathlib

import torch

import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
# CONSTANTS
DATA_DIR = '~/pytorch_data/'
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
MNIST_SIZE = 28
NUM_EPOCHS = 2
LEARNING_RATE = 1e-7

trans = transforms.Compose([transforms.Grayscale(), transforms.Resize((MNIST_SIZE, MNIST_SIZE)), transforms.ToTensor()])
mnist_loader = torch.utils.data.DataLoader(datasets.MNIST(root=DATA_DIR, train=True, download=True, 
                                                          transform=trans), shuffle=True)

mnist_loader = DataLoader(datasets.MNIST(root=DATA_DIR, train=True, download=True, transform=trans), shuffle=True)

# About AE's
Auto-Encoders work in encoder, $e$, decoder, $d$ pairs. The goals of these 2 components is to
- Maximize the amount of information held while encoding, $e^*$
- Minimize the amount of information lost while decoding, $d^*$

We express this function as
$$ e^*, d^* = \arg \min \epsilon\left(x, d(e(x))\right)$$
Where $d(e(x))$ is the reconstructed output

## Autoencoder

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        encoder_layers = [nn.Linear(in_features=MNIST_SIZE*MNIST_SIZE, out_features=512), nn.ReLU(),
                          nn.Linear(in_features=512, out_features=256), nn.ReLU(),
                          nn.Linear(in_features=256, out_features=128), nn.ReLU()]
        self.encoder = nn.Sequential(*encoder_layers)
        
        decoder_layers = [nn.Linear(in_features=128, out_features=256), nn.ReLU(),
                          nn.Linear(in_features=256, out_features=512), nn.ReLU(),
                          nn.Linear(in_features=512, out_features=MNIST_SIZE*MNIST_SIZE), nn.ReLU()]
        self.decoder = nn.Sequential(*decoder_layers)
    
    def forward(self, x):
        return self.decoder(self.encoder(x))

In [ ]:
def train():
    # architecture
    net = Autoencoder()
    net = net.to(DEVICE)
    
    # loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

    # training loop
    train_loss = []
    for epoch in range(NUM_EPOCHS):
        running_loss = 0.0
        for data in mnist_loader:
            img, _ = data
            img = img.to(DEVICE)
            img = img.view(img.size(0), -1) # unravel image
            optimizer.zero_grad()
            outputs = net(img)
            loss = criterion(outputs, img)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        loss = running_loss / len(mnist_loader)
        train_loss.append(loss)
        print('Epoch {} of {}, Train Loss: {:.3f}'.format(epoch+1, NUM_EPOCHS, loss))
    return net


In [ ]:
net = train()

## Results EDA

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
img, label = next(iter(mnist_loader))
print(label)
plt.imshow(img[0][0].cpu(), cmap='gray', interpolation='none')

In [ ]:
img = img.to(DEVICE)
img = img.view(img.size(0), -1)
net.encoder(img)

In [ ]:
img = img.to(DEVICE)
img = img.view(img.size(0), -1)
res = net(img).view(MNIST_SIZE, MNIST_SIZE)

In [ ]:
plt.imshow(res.cpu().detach().numpy(), cmap='gray', interpolation='none')